In [1]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.nn import functional as F
import torchvision.models
import torchvision.transforms as transforms
from options.train_options import TrainOptions
import extract_gan as E
import sys; sys.argv=['']; del sys

In [2]:
# %tb 
opt = TrainOptions().parse()

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 1                             
                 fineSize: 256                           
                  gpu_ids: -1 

In [3]:
# opt.init_type

In [3]:
model = E.ExtractGANModel(opt)

VGG parameters loaded.
Encoder weights initialized using xavier.
Decoder weights initialized using xavier.
StyleExtractor weights initialized using xavier.
StyleWhitener weights initialized using xavier.
Generator build success!
Discriminator weights initialized using xavier.
New ExtractGAN model initialized!


In [4]:
model.save_networks('test')

In [5]:
model.train()
# model.eval()

In [7]:
# model.D.vgg16.training
model.D.training

True

In [8]:
model.vgg16 is model.G.style_extractor.vgg16

True

In [9]:
model.vgg16 is model.D.vgg16

True

In [4]:
a = torch.randn(2,3,224,224)
a.shape

torch.Size([2, 3, 224, 224])

In [5]:
b = torch.randn(2,3,224,224)
b.shape

torch.Size([2, 3, 224, 224])

In [6]:
model.G.encoder(b)

tensor([[[[0.3813, 1.1244, 0.3706,  ..., 1.3001, 0.0000, 0.3424],
          [0.0000, 0.1954, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.4038,  ..., 0.7677, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.1458,  ..., 0.4535, 0.3362, 0.0000],
          [0.0000, 2.0069, 0.1050,  ..., 0.4651, 0.7821, 0.4299],
          [0.0000, 0.6532, 0.0000,  ..., 0.0000, 0.8350, 1.1915]],

         [[0.0000, 0.4780, 0.2852,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.3851, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.8612,  ..., 0.0000, 1.3642, 0.0000],
          ...,
          [0.0000, 0.9385, 0.0000,  ..., 0.0000, 0.9511, 0.0000],
          [0.5273, 0.0000, 0.9407,  ..., 0.0000, 1.0693, 0.0000],
          [0.0000, 1.3836, 1.0334,  ..., 0.4107, 0.0000, 0.0000]],

         [[0.2117, 0.0000, 0.1687,  ..., 0.0000, 0.2691, 0.0000],
          [0.0552, 1.3143, 0.2736,  ..., 0.5335, 0.1512, 0.0000],
          [0.0000, 0.0000, 0.3128,  ..., 0

In [7]:
model.G.style_whitener(model.G.encoder(b))

tensor([[[[1.0843, 2.8186, 1.5721,  ..., 1.5057, 0.6868, 1.4935],
          [0.6665, 0.4466, 1.1388,  ..., 0.5569, 0.7987, 1.1575],
          [1.2511, 0.7572, 0.7981,  ..., 0.0000, 0.0000, 0.5665],
          ...,
          [0.0901, 0.2814, 1.8620,  ..., 0.0000, 0.8706, 1.4772],
          [0.8673, 0.2179, 0.3469,  ..., 0.1622, 0.2307, 2.1731],
          [0.0000, 0.8637, 0.5993,  ..., 1.8603, 2.3138, 3.3964]],

         [[0.2527, 1.1208, 0.5286,  ..., 0.9537, 0.8011, 0.0193],
          [0.0356, 0.0000, 0.0000,  ..., 0.0624, 0.9815, 0.0000],
          [0.0000, 0.0000, 2.2659,  ..., 0.0000, 0.0000, 1.6743],
          ...,
          [0.0000, 1.3723, 0.0000,  ..., 0.9386, 1.8166, 0.8673],
          [0.5845, 1.7449, 3.9501,  ..., 0.0000, 0.0000, 0.0000],
          [0.9457, 2.7981, 1.7807,  ..., 0.0000, 0.0000, 0.2723]],

         [[0.2420, 0.0038, 0.1271,  ..., 0.0000, 1.2484, 0.0000],
          [0.0000, 3.3150, 0.6591,  ..., 1.2887, 0.8218, 0.0000],
          [0.0000, 0.9946, 0.5083,  ..., 0

In [50]:
model.G.style_extractor(b)[0].shape

torch.Size([2, 64, 64, 3, 3])

In [42]:
b.size(0)

2

In [8]:
model.G(a,b)

tensor([[[[-1.3509e-02, -1.3509e-02, -1.3509e-02,  ..., -1.3509e-02,
           -1.3509e-02, -1.3509e-02],
          [-1.3509e-02,  5.2182e-01,  1.9529e-01,  ..., -2.5223e-01,
            1.2259e-01, -1.3509e-02],
          [-1.3509e-02,  5.0047e-01, -4.9435e-01,  ..., -7.3626e-01,
           -1.9982e-01, -1.3509e-02],
          ...,
          [-1.3509e-02,  2.8727e-01,  4.5439e-01,  ..., -3.2707e-01,
           -5.1169e-01, -1.3509e-02],
          [-1.3509e-02,  3.1356e-01,  7.3350e-01,  ...,  3.5619e-01,
           -2.3479e-01, -1.3509e-02],
          [-1.3509e-02, -1.3509e-02, -1.3509e-02,  ..., -1.3509e-02,
           -1.3509e-02, -1.3509e-02]],

         [[-6.6207e-03, -6.6207e-03, -6.6207e-03,  ..., -6.6207e-03,
           -6.6207e-03, -6.6207e-03],
          [-6.6207e-03,  2.1237e-01,  3.2666e-01,  ...,  5.4076e-02,
           -6.9188e-02, -6.6207e-03],
          [-6.6207e-03,  8.0762e-02, -2.3873e-01,  ..., -5.0907e-02,
           -1.9203e-01, -6.6207e-03],
          ...,
     

In [9]:
model.D(a,b)

tensor([[0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)

In [59]:
a.shape

torch.Size([2, 3, 64, 64])

In [62]:
(a-mean)/std

tensor([[[[ 2.3989e+00, -6.7204e+00,  7.2132e+00,  ..., -5.9408e+00,
           -1.2460e+01, -6.3582e+00],
          [ 3.9324e+00,  8.6642e-01, -6.8124e+00,  ...,  6.3708e+00,
            4.5303e+00,  2.4185e+00],
          [ 6.6239e-01, -1.9510e+00, -5.1890e+00,  ..., -2.6839e+00,
           -5.3858e+00, -6.0654e+00],
          ...,
          [ 2.9787e+00, -9.5114e+00, -7.7846e-01,  ..., -2.6961e+00,
           -2.2449e+00, -4.9324e+00],
          [ 2.1153e-01,  2.1305e+00, -3.6233e+00,  ..., -4.9931e+00,
           -1.8343e+00, -6.9682e+00],
          [-8.8566e-01, -1.8230e+00,  1.8203e+00,  ...,  9.6630e+00,
            8.2382e+00, -3.1041e+00]],

         [[-4.7079e+00, -2.4070e+00, -4.6795e+00,  ...,  1.5349e-01,
           -6.6553e+00,  4.1584e+00],
          [-4.0344e+00, -8.7326e+00,  3.2987e+00,  ..., -6.8027e+00,
           -8.1533e-01,  2.3548e-01],
          [-7.0116e+00, -5.8293e-01,  3.8684e+00,  ...,  7.2226e-01,
           -6.8445e+00,  4.8851e-01],
          ...,
     

In [63]:
tt = torch.randn(2,3,4,5,3)

In [65]:
print((i.shape for i in tt))

<generator object <genexpr> at 0x7f1f0c590ed0>


In [76]:
t1 = torch.randn(5,2,3)

In [77]:
t1.shape

torch.Size([5, 2, 3])

In [78]:
t1.unsqueeze(0).shape

torch.Size([1, 5, 2, 3])

In [79]:
t1.shape

torch.Size([5, 2, 3])

In [82]:
torch.cat((t1.unsqueeze(0),t1.unsqueeze(0)), 0).shape

torch.Size([2, 5, 2, 3])

In [8]:
import time

In [9]:
time.time()

1543642276.244461

In [10]:
time.localtime()

time.struct_time(tm_year=2018, tm_mon=12, tm_mday=1, tm_hour=0, tm_min=31, tm_sec=22, tm_wday=5, tm_yday=335, tm_isdst=0)

In [11]:
time.gmtime()

time.struct_time(tm_year=2018, tm_mon=12, tm_mday=1, tm_hour=5, tm_min=32, tm_sec=10, tm_wday=5, tm_yday=335, tm_isdst=0)

In [17]:
time.strftime("%Y-%m-%d-%H-%M-%S_UTC", time.gmtime()) 

'2018-12-01-05-34-23_UTC'